## Loading necessary libraries

In [20]:
import pandas as pd, numpy as np, pickle
from interactiontransformer.InteractionTransformer import InteractionTransformer, run_shap
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from imblearn.ensemble import BalancedRandomForestClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib as mpl
import scipy
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings("ignore")
mpl.rcParams['figure.dpi'] = 300
sns.set(style='white',font_scale=0.5)

## Loading the data

In [21]:
df=pd.read_csv('../test_data/epistasis.test.csv')
X,y=df.iloc[:,:-1],df.iloc[:,-1]

## Train Test Split

In [22]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42,stratify=y,shuffle=True)

## Find the top interactions in the training set

In [23]:
transformer=InteractionTransformer(max_train_test_samples=1000,mode_interaction_extract=int(np.sqrt(X_train.shape[1]))) # mode_interaction_extract='sqrt'
transformer.fit(X_train,y_train)

## Transform design matrices for training and test sets

In [24]:
X_train2=transformer.transform(X_train)
X_test2=transformer.transform(X_test)

## Extract top ranked interactions via SHAP

In [25]:
transformer.all_interaction_shap_scores.sort_values('shap_interaction_score',ascending=False).iloc[:10]

,feature_1,feature_2,shap_interaction_score
1532,P1_1,P2_1,0.027191
1531,P1_1,P2_0,0.022998
1528,P1_0,P2_1,0.018854
1527,P1_0,P2_0,0.017997
1238,N11_1,P2_1,0.004296
712,N5_0,N17_1,0.003068
1229,N11_1,N16_1,0.003035
1206,N11_0,N17_0,0.003008
420,N3_0,N6_0,0.002960
725,N5_1,N7_1,0.002794


## Fit Models and then get AUROC

In [26]:
lr=LogisticRegression(random_state=42,class_weight='balanced').fit(X_train,y_train)
lr2=LogisticRegression(random_state=42,class_weight='balanced').fit(X_train2,y_train)
rf=BalancedRandomForestClassifier(random_state=42).fit(X_train,y_train)

print(roc_auc_score(y_test,lr.predict_proba(X_test)[:,-1]))
print(roc_auc_score(y_test,lr2.predict_proba(X_test2)[:,-1]))
print(roc_auc_score(y_test,rf.predict_proba(X_test)[:,-1]))

0.47324999999999995
0.8470750000000001
0.7993


## Collect SHAP Feature Importances

In [27]:
shap_lr=run_shap(X_train, X_test, lr, model_type='linear', explainer_options={}, get_shap_values_options={}, overall=False, savefile='../test_data/epistasis.lr.shap.png')

In [28]:
shap_rf=run_shap(X_train, X_test, rf, model_type='tree', explainer_options={}, get_shap_values_options={}, overall=False, savefile='../test_data/epistasis.rf.shap.png')

In [29]:
shap_lr2=run_shap(X_train2, X_test2, lr2, model_type='linear', explainer_options={}, get_shap_values_options={}, overall=False, savefile='../test_data/epistasis.lr2.shap.png')
